In [18]:
import FINE as fn
from getData import getData
import pandas as pd
data = getData()

%load_ext autoreload
%autoreload 2
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
locations = data['locations']
commodityUnitDict = {'electricity': 'kWh_el', 'hydrogen': 'kWh_H2_LHV'}
commodities = {'electricity','hydrogen'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [20]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='€', lengthUnit='m', verboseLogLevel=2)

In [21]:
esM.add(fn.Source(esM=esM, name='Electricity purchase', commodity='electricity', hasCapacityVariable=False,
                  operationRateMax=data['El Purchase, operationRateMax'], commodityCost=0.30))

In [22]:
esM.add(fn.Source(esM=esM,
                  name='PV',
                  commodity='electricity',
                  hasCapacityVariable=True,
                  hasIsBuiltBinaryVariable=True,
                  operationRateMax=data['PV, operationRateMax'], 
                  capacityMax=data['PV, capacityMax'],
                  interestRate = 0.04,
                  economicLifetime = 20,
                  investIfBuilt=1000,
                  investPerCapacity=1400,
                  opexIfBuilt = 10,
                  bigM = 40))

In [23]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit='kWh_el',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.04,
                      economicLifetime=10))

In [24]:
esM.add(fn.Conversion(esM=esM, name='Fuel Cell', physicalUnit='kWh_H2_LHV',
                      commodityConversionFactors={'electricity':1,'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.04,
                      economicLifetime=10))

In [44]:
esM.add(fn.Storage(esM=esM,
                   name='Battery Storage',
                   commodity='electricity',
                   cyclicLifetime=10000,
                   chargeEfficiency=0.95,
                   dischargeEfficiency=0.95,
                   chargeRate=0.5,
                   dischargeRate=0.5,
                   hasIsBuiltBinaryVariable=True,
                   capacityMax=data['BS, capacityMax'],
                   interestRate = 0.04,
                   economicLifetime = 12,
                   investIfBuilt=2000,
                   investPerCapacity=100,
                   bigM = 110))

In [45]:
esM.add(fn.Storage(esM=esM, name='hydrogen tank', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=3000,
                   chargeRate=1/470.37, dischargeRate=1/470.37,
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['HS, capacityMax'],
                   investPerCapacity=110, opexPerCapacity=57, interestRate=0.04,
                   economicLifetime=30))

In [46]:
esM.add(fn.Transmission(esM=esM,
                        name='E_Distribution_Grid',
                        commodity='electricity',
                        losses=0.00001,
                        distances = data['cables, distances'],
                        capacityFix=data['cables, capacityFix']))

In [47]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

In [48]:
esM.cluster(numberOfTypicalPeriods=7)

C:\ProgramData\Anaconda3\lib\site-packages\FINE\energySystemModel.py:372: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  timeSeriesData = timeSeriesData.reindex_axis(sorted(timeSeriesData.columns), axis=1)


In [49]:
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='cuts=0 method=2')

Academic license - for non-commercial use only
Read LP format model from file C:\Users\Derek\python files\tmp8zeb8snb.pyomo.lp
Reading time = 0.14 seconds
x18619: 33159 rows, 18619 columns, 94348 nonzeros
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


In [50]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2)

grid1 grid2 grid3 grid4 grid5  \
Component            Property     Unit                                         
Electricity demand   operation    [kWh_el*h/a]   NaN   NaN   NaN   NaN   NaN   
Electricity purchase TAC          [€/a]            0     0     0     0     0   
                     commodCosts  [€/a]          NaN   NaN   NaN   NaN   NaN   
                     operation    [kWh_el*h/a]   NaN   NaN   NaN   NaN   NaN   
PV                   TAC          [€/a]            0     0     0     0     0   
                     capacity     [kWh_el]       NaN   NaN   NaN   NaN   NaN   
                     capexCap     [€/a]          NaN   NaN   NaN   NaN   NaN   
                     capexIfBuilt [€/a]          NaN   NaN   NaN   NaN   NaN   
                     invest       [€]            NaN   NaN   NaN   NaN   NaN   
                     isBuilt      [-]            NaN   NaN   NaN   NaN   NaN   
                     operation    [kWh_el*h/a]   NaN   NaN   NaN   NaN   NaN   
                     opexIfBuilt  [€/a]          NaN   NaN   NaN   NaN   NaN   

                                               grid6    home1    home2  \
Component            Property     Unit                                   
Electricity demand   operation    [kWh_el*h/a]   NaN    24719  19409.3   
Electricity purchase TAC          [€/a]            0        0        0   
                     commodCosts  [€/a]          NaN      NaN      NaN   
                     operation    [kWh_el*h/a]   NaN      NaN      NaN   
PV                   TAC          [€/a]            0   1916.4   1738.5   
                     capacity     [kWh_el]       NaN  17.8889  16.1619   
                     capexCap     [€/a]          NaN  1842.82  1664.91   
                     capexIfBuilt [€/a]          NaN  73.5818  73.5818   
                     invest       [€]            NaN  26044.5  23626.7   
                     isBuilt      [-]            NaN        1        1   
                     operation    [kWh_el*h/a]   NaN  16066.2  14573.5   
                     opexIfBuilt  [€/a]          NaN       10       10   

                                                  home3    home4    home5  \
Component            Property     Unit                                      
Electricity demand   operation    [kWh_el*h/a]  24892.8  17439.8    23310   
Electricity purchase TAC          [€/a]               0        0        0   
                     commodCosts  [€/a]             NaN      NaN      NaN   
                     operation    [kWh_el*h/a]      NaN      NaN      NaN   
PV                   TAC          [€/a]         1853.36  1286.21  1765.98   
                     capacity     [kWh_el]       17.277  11.7714  16.4288   
                     capexCap     [€/a]         1779.78  1212.63   1692.4   
                     capexIfBuilt [€/a]         73.5818  73.5818  73.5818   
                     invest       [€]           25187.8    17480  24000.3   
                     isBuilt      [-]                 1        1        1   
                     operation    [kWh_el*h/a]  15146.3  8487.17  11871.3   
                     opexIfBuilt  [€/a]              10       10       10   

                                               transformer  
Component            Property     Unit                      
Electricity demand   operation    [kWh_el*h/a]         NaN  
Electricity purchase TAC          [€/a]            15033.7  
                     commodCosts  [€/a]            15033.7  
                     operation    [kWh_el*h/a]     50112.4  
PV                   TAC          [€/a]                  0  
                     capacity     [kWh_el]             NaN  
                     capexCap     [€/a]                NaN  
                     capexIfBuilt [€/a]                NaN  
                     invest       [€]                  NaN  
                     isBuilt      [-]                  NaN  
                     operation    [kWh_el*h/a]         NaN  
                 

In [51]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

,,,grid1,grid2,grid3,grid4,grid5,grid6,home1,home2,home3,home4,home5,transformer
Component,Property,Unit,,,,,,,,,,,,


In [52]:
esM.getOptimizationSummary("StorageModel", outputLevel=2)

grid1 grid2 grid3 grid4 grid5  \
Component       Property           Unit                                         
Battery Storage TAC                [€/a]            0     0     0     0     0   
                capacity           [kWh_el*h]     NaN   NaN   NaN   NaN   NaN   
                capexCap           [€/a]          NaN   NaN   NaN   NaN   NaN   
                capexIfBuilt       [€/a]          NaN   NaN   NaN   NaN   NaN   
                invest             [€]            NaN   NaN   NaN   NaN   NaN   
                isBuilt            [-]            NaN   NaN   NaN   NaN   NaN   
                operationCharge    [kWh_el*h/a]   NaN   NaN   NaN   NaN   NaN   
                operationDischarge [kWh_el*h/a]   NaN   NaN   NaN   NaN   NaN   

                                                grid6 home1    home2  \
Component       Property           Unit                                
Battery Storage TAC                [€/a]            0     0  1278.63   
                capacity           [kWh_el*h]     NaN     0      100   
                capexCap           [€/a]          NaN     0  1065.52   
                capexIfBuilt       [€/a]          NaN     0  213.104   
                invest             [€]            NaN     0    12000   
                isBuilt            [-]            NaN     0        1   
                operationCharge    [kWh_el*h/a]   NaN     0  17388.8   
                operationDischarge [kWh_el*h/a]   NaN     0  15693.4   

                                                       home3 home4    home5  \
Component       Property           Unit                                       
Battery Storage TAC                [€/a]                   0     0  971.822   
                capacity           [kWh_el*h]              0     0  71.2062   
                capexCap           [€/a]                   0     0  758.718   
                capexIfBuilt       [€/a]                   0     0  213.104   
                invest             [€]                     0     0  9120.62   
                isBuilt            [-]                     0     0        1   
                operationCharge    [kWh_el*h/a]            0     0  15889.8   
                operationDischarge [kWh_el*h/a]  1.94404e-11     0  14340.6   

                                                transformer  
Component       Property           Unit                      
Battery Storage TAC                [€/a]                  0  
                capacity           [kWh_el*h]           NaN  
                capexCap           [€/a]                NaN  
                capexIfBuilt       [€/a]                NaN  
                invest             [€]                  NaN  
                isBuilt            [-]                  NaN  
                operationCharge    [kWh_el*h/a]         NaN  
                operationDischarge [kWh_el*h/a]         NaN